In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

import joblib


In [3]:
# Importing data
df = pd.read_csv('data/labelencoded_data.csv',index_col=0)


In [4]:
# Importing data
df = pd.read_csv('data/labelencoded_data.csv',index_col=0)
# df.drop(['id', 'std', 'sta', 'fltid','arr_iata','dep_iata','ac'], axis=1,inplace=True)


In [5]:
#Making a preprocessing pipeline

# Data loading
df_flightdata = pd.read_csv('data/Train.csv')
df_airportdata = pd.read_csv('data/airportdata.csv', index_col=0)

# Defining categorical data and features to drop. Here we use label encoding. Hot encoding is not used.
columns_to_drop = ['id', 'std', 'sta', 'fltid', 'arr_iata', 'dep_iata', 'ac','status']
categorical_columns = ['depstn', 'arrstn',  'arr_country', 'dep_country', 'season', 'airline_code', 'international_flight']

# Join the DataFrames first and then apply the preprocessing pipeline
df = df_flightdata.join(df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']].add_prefix('dep_'), how='left', on='DEPSTN') \
    .join(df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']].add_prefix('arr_'), how='left', on='ARRSTN')
    
display(merged_data.T)

,0,1,2,3,4,5,6,7,8,9,...,107823,107824,107825,107826,107827,107828,107829,107830,107831,107832
ID,train_id_0,train_id_1,train_id_2,train_id_3,train_id_4,train_id_5,train_id_6,train_id_7,train_id_8,train_id_9,...,train_id_107823,train_id_107824,train_id_107825,train_id_107826,train_id_107827,train_id_107828,train_id_107829,train_id_107830,train_id_107831,train_id_107832
DATOP,2016-01-03,2016-01-13,2016-01-16,2016-01-17,2016-01-17,2016-01-17,2016-01-18,2016-01-18,2016-01-18,2016-01-18,...,2018-04-12,2018-02-09,2018-08-08,2018-10-15,2018-12-19,2018-07-05,2018-01-13,2018-11-07,2018-01-23,2018-11-13
FLTID,TU 0712,TU 0757,TU 0214,TU 0480,TU 0338,TU 0283,TU 0514,TU 0716,TU 0752,TU 0996,...,TU 9001,UG 1730,WKL 0000,UG 0011,SGT 0000,WKL 0000,UG 0003,SGT 0000,UG 0010,UG 0002
DEPSTN,CMN,MXP,TUN,DJE,TUN,TLS,TUN,TUN,TUN,TUN,...,MIR,TUN,TUN,DJE,TUN,TUN,DJE,TUN,TUN,TUN
ARRSTN,TUN,TUN,IST,NTE,ALG,TUN,BCN,ORY,FCO,NCE,...,TUN,NAP,TUN,TUN,TUN,TUN,TUN,TUN,DJE,DJE
STD,2016-01-03 10:30:00,2016-01-13 15:05:00,2016-01-16 04:10:00,2016-01-17 14:10:00,2016-01-17 14:30:00,2016-01-17 16:20:00,2016-01-18 07:15:00,2016-01-18 07:35:00,2016-01-18 07:40:00,2016-01-18 07:45:00,...,2018-04-12 17:05:00,2018-02-09 10:00:00,2018-08-08 22:00:00,2018-10-15 19:45:00,2018-12-19 23:45:00,2018-07-05 23:00:00,2018-01-13 08:00:00,2018-11-07 05:00:00,2018-01-23 18:00:00,2018-11-13 06:15:00
STA,2016-01-03 12.55.00,2016-01-13 16.55.00,2016-01-16 06.45.00,2016-01-17 17.00.00,2016-01-17 15.50.00,2016-01-17 18.15.00,2016-01-18 09.00.00,2016-01-18 09.55.00,2016-01-18 09.00.00,2016-01-18 09.15.00,...,2018-04-12 17.40.00,2018-02-09 11.15.00,2018-08-09 01.00.00,2018-10-15 20.45.00,2018-12-20 01.45.00,2018-07-06 02.00.00,2018-01-13 09.00.00,2018-11-07 12.50.00,2018-01-23 18.45.00,2018-11-13 07.05.00
STATUS,ATA,ATA,ATA,ATA,ATA,ATA,ATA,ATA,ATA,ATA,...,ATA,SCH,SCH,SCH,SCH,SCH,SCH,SCH,ATA,SCH
AC,TU 32AIMN,TU 31BIMO,TU 32AIMN,TU 736IOK,TU 320IMU,TU 736IOP,TU 32AIMH,TU 32AIMI,TU 32AIMC,TU 31AIMK,...,TU 736IOL,TU CR9ISA,TU 320IMV,UG AT7LBE,TU 736IOP,TU 32AIML,UG AT7AT7,TU 736IOK,TU CR9ISA,TU CR9ISA
target,260.0,20.0,0.0,0.0,22.0,53.0,10.0,15.0,16.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,dep_iata,dep_country,dep_elevation,dep_lat,dep_lon,arr_iata,arr_country,arr_elevation,arr_lat,arr_lon
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0,CMN,MA,656.0,33.367500,-7.58997,TUN,TN,22.0,36.851002,10.22720
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0,MXP,IT,768.0,45.630600,8.72811,TUN,TN,22.0,36.851002,10.22720
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0,TUN,TN,22.0,36.851002,10.22720,IST,TR,325.0,41.275333,28.75200
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0,DJE,TN,19.0,33.875000,10.77550,NTE,FR,90.0,47.153198,-1.61073
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0,TUN,TN,22.0,36.851002,10.22720,ALG,DZ,82.0,36.691002,3.21541


In [8]:
# Splitting data
# X = df[['arrstn','depstn','std','sta']]
X = df.drop(['target'],axis=1)
y = df['target'] 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [12]:
X.head()

,arrstn,depstn,std_time,sta_time,flight_time_in_min,datop
0,119,31,103000,125500,145.0,20160103
1,119,86,150500,165500,110.0,20160113
2,58,123,41000,64500,155.0,20160116
3,91,37,141000,170000,170.0,20160117
4,4,123,143000,155000,80.0,20160117


In [9]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


RMSE: 128.43504471511554


/home/pclinux/spiced/data_science/flight_delays_ML/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE: 128.43504471511554


In [10]:
# Save the model to a file
model_filename = './data/api_test_logistic_regression_model.joblib'
joblib.dump(lr, model_filename)

['./data/api_test_logistic_regression_model.pkl']